# Test PEP solver using refiniments on several codes 

With this notebook, you can test the PEP solver using squared hulls on several random weakly self-dual codes.

As a PEP solver, we rely on the GIP solver we describe in the paper.

For simplicity, we only do the full attack when hulls are trivial.

Combinements are combined with a meet-in-the-middle strategy.

In [7]:
reset()
load('code_utils.sage')
load('ABL_sample_weakly_self_dual.sage')
load("GIP_solver.sage")

In [13]:
def sample_PEP_instance(q, n, k, G):
    '''
    Sample random PEP instance: on S*G*P with random change of basis S, random permutation P
    '''
    Fq = GF(q) #finite field
    
    #sample change of basis
    rank_S = 0
    while rank_S < k:
        S = random_matrix(Fq, k, k)
        rank_S = rank(S)
    
    #sample random permutation
    P_as_list = Permutations(n).random_element()
    P = P_as_list.to_matrix().change_ring(Fq)
                
    #compute G'
    G_prime = S*G*P

    return G_prime, P

### Functions to use refinements

In [14]:
def matrix_count_in_Fq(n, Fq_list, A):
    '''
    Count how many times an element appears in a matrix
    '''
    vals = vector(ZZ, len(Fq_list))
    for ell in range(n):
        for i in range(n):
            j = Fq_list.index(A[ell, i])
            vals[j] += 1
    return str(vals)

def puncture_matrix(Fq, G, pos):
    '''
    Remove from G the columns indexed by pos
    '''
    k = G.nrows()
    n = G.ncols()
    t = len(pos)
    
    #puncture G in the positions indexed by pos
    new_G = matrix(Fq, k, n-t)
    num = 0
    for j in range(n):
        if j not in pos:
            new_G[:,num] = G[:,j]
            num+= 1
    return new_G

def prepare_first_list(Fq, G, t):
    '''
    Prepare first list
    '''
    k = G.nrows(); n = G.ncols()
    
    all_pos = Combinations(n,t)

    #prepare first list with lex ordering of obtained graph
    L1 = defaultdict(list)
    for pos in all_pos:

        #Puncture G in position indexed by pos
        new_G1 = puncture_matrix(Fq, G, pos)

        #Compute hull and squared hull
        U1 = hull(Fq, new_G1)
        M1 = square_code(U1) #basis of square of hull for first code

        if (rank(M1)-rank(M1*M1.transpose())) == 0: #continue only if squared hull has trivial hull
            
            A1 = M1.transpose()*(M1*M1.transpose())^-1*M1 #compute graph
            label = matrix_count_in_Fq(n-t, Fq_list, A1) #label associated with graph
           
            if L1.get(label) == None:
                L1[label] = [pos]
            else:
                L1[label].append(pos)

    return L1


def solver_with_refiniments(Fq, G, G_prime, t):
    '''
    This is our solver using refiniments. The number of positions which 
    are used for puncturation is t
    '''
    n = G.ncols(); k = G.nrows(); Fq_list = Fq.list()

    #Compute first list
    L1 = prepare_first_list(Fq, G, t)

    #compute elements of second list
    for pos in Combinations(n,t):

        #puncture
        new_G2 = puncture_matrix(Fq, G_prime, pos)

        #build graph
        U2 = hull(Fq, new_G2)
        M2 = square_code(U2) #basis of square of hull for first code

        if (rank(M2)-rank(M2*M2.transpose())) == 0: #continue only if squared hull has trivial hull
            
            A2 = M2.transpose()*(M2*M2.transpose())^-1*M2 #compute graph
            label = matrix_count_in_Fq(n-t, Fq_list, A2) #label associated with graph

            #See if label exists in L1. If it exists, solve GIP on graphs for punctured
            target_pos = L1.get(label)
           
            if target_pos != None:
                if t == 3:
                    print(pos,"collision found!")
                dim2 = rank(new_G2)
                
                for pos1 in target_pos:
                #    print(pos1)    
                    #Puncture and reconstruct graph
                    new_G1 = puncture_matrix(Fq, G, pos1)
                    dim1 = rank(new_G1)

                    if (dim1 == k)&(dim2 == k):
                    #    print("----> Building graphs")
                        
                        #Build graph
                        U1 = hull(Fq, new_G1)
                        M1 = square_code(U1) #basis of square of hull for first code
            
                        #Try to match columns
                        A1 = M1.transpose()*(M1*M1.transpose())^-1*M1 #compute graph   
    
                        #Solve with GIP
                        colls = simple_solve_GIP(Fq, A1, A2)    
                   #     print("------>",colls)
                        
                        #Permute matrix G1
                        perm_new_G1 = matrix(Fq, k, n-t)
                        failure = 0
                        for i in range(n-t):
                       #     print(colls[i])
                            if (colls[i][0] == None)|(colls[i][1] == None):
                                failure = 1
                                break
                            else:
                                perm_new_G1[:, colls[i][1]] = new_G1[:, colls[i][0]]

                        if failure:
                            break
    
                        #Find information set
   #                     print("searching for info sets")
   #                     print(perm_new_G1)
                        if rank(perm_new_G1)<k:
                            break

                        flag_ok = 0
                        while flag_ok == 0:
                            J = Combinations(n-t,k).random_element()
                            if (rank(perm_new_G1[:,J]) == k):
                                flag_ok = 1
    #                    print("search done")
                        #Compute change of basis
                        if (rank(new_G2[:, J]) == k):                            
                            S = new_G2[:,J]*perm_new_G1[:,J]^-1
    
                            #See if change of basis is ok for initial codes as well
                            same_basis_G1 = S*G
    
                            cols1 = [str(same_basis_G1[:,i]) for i in range(n)]
                            my_P = matrix(Fq, n, n)
 
                            for i in range(n):
                                target_col = str(G_prime[:,i])
                                try:
                                    j = cols1.index(target_col)
                                    my_P[j,i] = 1
                                except:
                                    break
                         #   print("---------------")
                         #   print(my_P)
                         #   print("---------------")
                            
                            #See if P is a permutation matrix
                            sum_rows = [sum(vector(my_P[i,:])) for i in range(n)]
                            sum_cols = [sum(vector(my_P[:,i])) for i in range(n)]
                         #   print(sum_rows)
                         #   print(sum_cols)
                            if (sum_rows.count(Fq(1)) == n) & (sum_cols.count(Fq(1)) == n):
                                return my_P
    return -1

# Single $n$ and $q$, seveal values of $h$

We fix values of $q$ and $n$, test several values of $h$

In [17]:
n = 30
q = 13
num_codes = 100

Fq = GF(q)
Fq_list = Fq.list()

vals_trivial_hull = []
vals_perm_ok = []
vals_perm_ok_refiniments_1 = []

for h in range(3, floor(sqrt(2*n)-0.5)+1):
    
    k = h

    #data for simulation
    pr_trivial_hull = 0; pr_P_ok = 0; pr_P_ok_1 = 0; pr_P_ok_2 = 0

    for id_code in range(num_codes):
       # print("#",id_code)
        
        #sample first code
        G = None
        while G == None:
            G = faster_sample_weakly_self_dual_code(q, n, h+1, h, False)
        G = hull(Fq, G)
    
        #sample PEP instance
        G_prime, P = sample_PEP_instance(q, n, k, G)
        
        ##square hulls
        U1 = square_code(G) #basis of square of hull for first code
        U2 = square_code(G_prime) #basis of square of hull for second code 

        #check if hull dim = 0
        hull_dim = rank(U1)-rank(U1*U1.transpose())
    
        if hull_dim == 0:
            pr_trivial_hull += 1
    
        #create graphs
        if hull_dim == 0:

            A1 = graph_from_generator_matrix(U1)
            A2 = graph_from_generator_matrix(U2)
        
            #solve GIP
            sol = simple_solve_GIP(Fq, A1, A2)
            
            #build permutation    
            my_P = matrix(Fq, n, n)
            for i in range(n):
            #    print(sol[i])
                j1 = sol[i][0]
                j2 = sol[i][1]
                my_P[j1, j2] = 1
            #print(my_P)
            #print(" ")
            
            #check if P is ok
            if my_P == P:
                pr_P_ok += 1
                pr_P_ok_1 += 1
            else:
                #test one refiniment
                my_P_1 = solver_with_refiniments(Fq, G, G_prime, 1)

                if my_P_1 == P:
                    pr_P_ok_1 += 1

        #Print results
        if (pr_trivial_hull != 0)&(id_code%20 == 0):
            print("h = ",h,", id = ",id_code,", Pr(h = 0) = ",N(pr_trivial_hull/(id_code+1.),5),", Pr(P found) = ",N(pr_P_ok/pr_trivial_hull,5),", Pr(P found, 1 ref) = ",N(pr_P_ok_1/pr_trivial_hull,5))

    vals_trivial_hull.append((h, N(pr_trivial_hull/num_codes)))
    vals_perm_ok.append((h, N(pr_P_ok/pr_trivial_hull)))
    vals_perm_ok_refiniments_1.append((h, N(pr_P_ok_1/pr_trivial_hull)))

h =  3 , id =  0 , Pr(h = 0) =  1.0 , Pr(P found) =  1.0 , Pr(P found, 1 ref) =  1.0
h =  3 , id =  20 , Pr(h = 0) =  0.84 , Pr(P found) =  0.62 , Pr(P found, 1 ref) =  0.72
h =  3 , id =  40 , Pr(h = 0) =  0.91 , Pr(P found) =  0.62 , Pr(P found, 1 ref) =  0.72
h =  3 , id =  60 , Pr(h = 0) =  0.94 , Pr(P found) =  0.69 , Pr(P found, 1 ref) =  0.78
h =  3 , id =  80 , Pr(h = 0) =  0.94 , Pr(P found) =  0.66 , Pr(P found, 1 ref) =  0.78
h =  4 , id =  0 , Pr(h = 0) =  1.0 , Pr(P found) =  1.0 , Pr(P found, 1 ref) =  1.0
h =  4 , id =  20 , Pr(h = 0) =  0.84 , Pr(P found) =  0.88 , Pr(P found, 1 ref) =  0.88
h =  4 , id =  40 , Pr(h = 0) =  0.88 , Pr(P found) =  0.91 , Pr(P found, 1 ref) =  0.94
h =  4 , id =  60 , Pr(h = 0) =  0.91 , Pr(P found) =  0.94 , Pr(P found, 1 ref) =  0.97
h =  4 , id =  80 , Pr(h = 0) =  0.94 , Pr(P found) =  0.94 , Pr(P found, 1 ref) =  0.97
h =  5 , id =  0 , Pr(h = 0) =  1.0 , Pr(P found) =  1.0 , Pr(P found, 1 ref) =  1.0
h =  5 , id =  20 , Pr(h = 0) =  

Write results

In [19]:
with open("Results_h/pr_trivial_hull_"+str(n)+"_"+str(q)+".txt", "w") as f:
    f.write("X Y\n")
    for x in vals_trivial_hull:
        f.write(str(x[0])+" "+str(x[1])+"\n")

with open("Results_h/pr_P_ok_"+str(n)+"_"+str(q)+".txt", "w") as f:
    f.write("X Y\n")
    for x in vals_perm_ok:
        f.write(str(x[0])+" "+str(x[1])+"\n")

with open("Results_h/pr_P_ok_1_"+str(n)+"_"+str(q)+".txt", "w") as f:
    f.write("X Y\n")
    for x in vals_perm_ok_refiniments_1:
        f.write(str(x[0])+" "+str(x[1])+"\n")